# CHAPTER 6 일급 함수 디자인 패턴

## 6.1 사례: 전략 패턴의 리팩토링
전략 패턴은 파이썬에서 함수를 일급 객체로 사용하면 더욱 간단해질 수 있는 디자인 패턴의 대표적인 사례다.

### 6.1.1 고전적인 전략

전략 패턴에 대한 UML 클래스 다이어그램에 들어가는 구성 요소  

**콘텍스트**  
일부 계산을 서로 다른 알고리즘을 구현하는 교환 가능한 컴포넌트에 위임함으로써 서비스를 제공한다. 전자상거래 예제에서 콘텍스트는 `Order`로서, 여러 알고리즘 중 하나에 따라 프로모션 할인을 적용하도록 설정된다.  

**전략**  
여러 알고리즘을 구현하는 컴포넌트에 공통된 인터페이스, 전자상거래 예제에서는 이 역할을 `Promotion`이라는 추상 클래스가 담당한다.  

**구체적인 전략**  
전략의 구상 서브클래스 중 하나, 여기서는 `FidelityPromo`, `BulkItemPromo`, `LargeOrderPromo` 등 총 3개의 구체적인 전략이 구현되어 있다.

**디자인 패턴**에서는 전략 패턴을 다음과 같이 설명한다.

* 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.  

온라인 상점의 할인 규칙
* 충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인을 적용한다.
* 하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인을 적용한다.
* 서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인을 적용한다.

In [2]:
# 예제 6-1 플러그형 할인 전략을 가진 Order 클래스 구현
# 디자인 패턴에 설명된 대로 구체적인 전략은 콘텍스트 클래스의 클라이언트에 의해 선택된다.
# 이 예제에서는 주문 객체를 생성하기 전에 시스템이 할인 전략을 선택해서 Order 생성자에 전달한다.
# 전략의 선택은 패턴 범위를 벗어난다.
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [3]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [12]:
class Order: # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) ################ 이 부분이 바뀔 것임.
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC): # 전략: 추상 베이스 클래스
    
    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환한다."""

In [6]:
class FidelityPromo(Promotion): # 첫 번째 구체적인 전략
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [7]:
class BulkItemPromo(Promotion): # 두 번째 구체적인 전략
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order):
        discount = 0 
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [21]:
class LargeOrderPromo(Promotion): # 세 번째 구체적인 전략
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [22]:
# 예제 6-2 여러 프로모션 할인을 적용해서 Order 클래스를 사용하는 예
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [23]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [24]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [25]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [26]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [27]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [28]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [29]:
Order(joe, cart, LargeOrderPromo())


<Order total: 42.00 due: 42.00>

### 6.1.2 함수지향 전략

`[예제6-1]`에서 각각의 구체적인 전략은 `discount()`라는 메서드 하나를 가진 클래스다.  
게다가 전략 객체는 상태(객체 속성)를 가지고 있지 않다. 구체적인 전략 객체가 일반 함수로 보여야 한다.  

In [30]:
# 예제 6-3 할인 전략을 함수로 구현한 Order 클래스
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [31]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [49]:
class Order: # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # 할인액을 계산하려면 self.promotion() 함수를 호출하면 된다.
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
# 추상클래스는 정의하지 않는다.

In [44]:
def fidelity_promo(order): # 각각의 구체적인 전략이 함수로 구현되었다.
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [45]:
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [46]:
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [47]:
# 예제 6-4 할인 전략을 함수로 정의한 Order 클래스의 사용 예
# 더 간단해졌다!
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [50]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [51]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [52]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [53]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [54]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [55]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [56]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

[예제6-4]에서는 Order 객체마다 할인 전략 객체를 만들 필요가 없다. 할인 전략 함수를 바로 사용할 수 있기 때문이다.  

* "전략 객체는 종종 훌륭한 플라이웨이트(flyweight)가 되기도 한다"는 흥미로운 설명을 했다.  
* "플라이웨이트는 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체"라고 정의했다.  
* 새로운 콘텍스트에서 동일 전략 객체를 반복해서 적용할 때는 새로 생성하는 비용을 줄이기 위해 플라이트웨이트를 공유하는 것이 좋다.
* 따라서 전략 패턴의 단점인 '런타임 비용'을 극복하기 위해 저자들은 또 다른 패턴인 플라이웨이트 패턴을 사용하도록 권고한다. 
* 반면 소스 코드의 행 수와 유지보수 비용은 눈덩이처럼 불어난다.

* 함수는 사용자 정의 클래스보다 훨씬 가볍고 파이썬이 모듈을 컴파일할 때 단 한번만 생성되므로 플라이웨이트가 필요하지 않다.
* 일반 함수도 '여러 콘텍스트에서 동시에 공유할 수 있는 공유 객체'임을 명심하자.

### 6.1.3 최선의 전략 선택하기: 단순한 접근법

In [58]:
# 예제 6-5 모든 할인을 적용해서 가장 큰 값을 반환하는 best_promo() 함수
Order(joe, long_order, best_promo)
Order(joe, banana_cart, best_promo)
Order(ann, cart, best_promo)

In [59]:
# 예제 6-6 함수 리스트를 반복해서 최대 할인액을 찾아내는 best_promo() 함수
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

[예제 6-6]은 가독성이 좋고 제대로 작동하지만, 일부 코드가 중복되어 있어 버그가 생길 여지가 있다.  
새로운 할인 전략을 추가하려면 함수를 코딩하고 이 함수를 promos 리스트에 추가해야 한다.  
아니면 새로운 할인 함수를 Order 객체에 인수로 전달해서 작동시킬 수 있지만, 이때 `best_promo()`는 새로운 할인 함수를 고려하지 않는다.

### 6.1.4 모듈에서 전략 찾기
파이썬 모듈도 일급 객체로서, 모듈을 다루는 여러 함수가 표준 라이브러리에서 제공된다. 파이썬 문서에서는 `globals()` 내장 함수를 다음과 같이 설명하고 있다.

**`globals()`**  
현재 전역 심벌 테이블을 나타내는 딕셔너리 객체를 반환한다. 이 딕셔너리는 언제나 현재 모듈에 대한 내용을 담고 있다(함수나 메서드 안에서 호출할 때, 함수를 호출한 모듈이 아니라 함수가 정의된 모듈을 나타낸다).

In [63]:
# 예제 6-7 모듈 전역 네임스페이스를 내부 조사해서 만든 promo 리스트
promos = [globals()[name] for name in globals()    # globals() 함수가 반환한 딕셔너리에서 name을 반복한다.
          if name.endswith('_promo')               # _promo로 끝나는 name만 선택한다,
          and name != 'best_promo']                # 무한 재귀 호출을 피하기 위해 best_promo 자신은 걸러낸다.

In [68]:
def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)    # best_promo() 내부는 변경되지 않았다.

In [70]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [71]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [72]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

[예제 6-8]에서는 promotions라는 별도 모듈에 내부 조사를 수행해서 만든 전략 함수 리스트를 사용하도록 변경했다. [예제 6-8]은 `inspect`와 `promotions`를 임포트해야 작동한다.  
`inspect`모듈은 상위 수준의 내부 조사 함수를 제공한다(여기서는 코드를 간단히 표현하기 위해 일반적으로 파일의 윗부분에서 수행하는 임포트 명령을 생략했다.)

In [74]:
# 예제 6-8 새로운 promotions 모듈을 내부 조사해서 만든 promos 리스트
import inspect

promos = [func for name, func in 
                    inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)    # best_promo() 내부는 변경되지 않았다.

`inspect.getmembers()` 함수는 조건식(predicate; 불리언형 함수가 사용된다)으로 걸러낸 객체의 속성들을 반환한다. 

## 6.2 명령
함수를 인수로 전달하는 기법을 사용하면 명령 디자인 패턴도 구현을 단순하게 만들 수 있다.

In [75]:
# 예제 6-9 각각의 MacroCommand 객체는 내부에 명령 리스트를 갖고 있다.
class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    
    def __init__(self, commands):
        self.commands = list(commands)
        # commands 인수로부터 리스트를 만들면 명령들이 반복 가능한 객체임이 보장되며,
        # 각각의 MacroCommand 객체 안에 명령에 대한 참조를 복사하게 된다.
        
    def __call__(self):
        for command in self.commands: 
        # MacroCommand 객체가 호출되면 self.commands에 들어있는 명령이 순서대로 호출된다.
            command()